<a href="https://colab.research.google.com/github/41371120h/PL-Repo.peng/blob/main/HW5_%E7%8E%A9%E6%A8%82_%E8%81%9A%E6%9C%83%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#試算表連結:https://docs.google.com/spreadsheets/d/1U5MQqini-y-jN7-eZXYId8l5VBlTs1LXgDJUNRVah3w/edit#gid=0

In [8]:
!pip install -q gspread gspread_dataframe pandas gradio google-generativeai folium geopy

import pandas as pd
import random
import asyncio
import gradio as gr
import google.generativeai as genai
import folium
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

# --- Google 帳號授權 ---
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# --- 讀取試算表 ---
sheet_url = "https://docs.google.com/spreadsheets/d/1U5MQqini-y-jN7-eZXYId8l5VBlTs1LXgDJUNRVah3w/edit#gid=0"
worksheet = gc.open_by_url(sheet_url).worksheet("玩樂地點")
df = get_as_dataframe(worksheet, evaluate_formulas=True).dropna(how="all").dropna(axis=1, how="all")

# --- 緯度/經度欄位檢查與補齊（以玩樂地點名稱查詢） ---
if '緯度' not in df.columns or '經度' not in df.columns:
    geolocator = Nominatim(user_agent="taipei_map")
    df['緯度'] = None
    df['經度'] = None
    for i, row in df.iterrows():
        try:
            # 以玩樂地點名稱查座標
            location = geolocator.geocode(f"{row['名稱']}, 台北市, 台灣")
            if location:
                df.at[i,'緯度'] = location.latitude
                df.at[i,'經度'] = location.longitude
        except:
            df.at[i,'緯度'] = df.at[i,'經度'] = None
        time.sleep(1)

# --- 模擬距離分類欄位（短、中、長） ---
if '距離' not in df.columns:
    df['距離'] = random.choices(['短','中','長'], k=len(df))

# --- Gemini 設定 ---
api_key = "YOUR_GEMINI_API_KEY"  # <- 請填入你的 Gemini API Key
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.5-pro')

# --- 計算使用者到各地點距離 ---
def compute_distance(user_location, df_locations):
    distances = []
    for _, row in df_locations.iterrows():
        if pd.notna(row['緯度']) and pd.notna(row['經度']):
            loc = (row['緯度'], row['經度'])
            distances.append(geodesic(user_location, loc).km)
        else:
            distances.append(float('inf'))
    return distances

# --- 過濾與推薦函式 ---
def filter_and_recommend(df, price_filter, distance_filter, user_addr=None):
    if not price_filter:
        price_filter = df['消費預算'].dropna().unique()
    if not distance_filter:
        distance_filter = df['距離'].unique()

    filtered_df = df[df['消費預算'].isin(price_filter) & df['距離'].isin(distance_filter)].copy()

    if user_addr:
        geolocator = Nominatim(user_agent="taipei_map")
        loc = geolocator.geocode(f"{user_addr}, 台北市, 台灣")
        if loc:
            user_location = (loc.latitude, loc.longitude)
            filtered_df['距離使用者(km)'] = compute_distance(user_location, filtered_df)
            distance_bins = []
            for d in filtered_df['距離使用者(km)']:
                if d < 2:
                    distance_bins.append('短')
                elif d < 5:
                    distance_bins.append('中')
                else:
                    distance_bins.append('長')
            filtered_df['距離分類自動'] = distance_bins
            filtered_df = filtered_df[filtered_df['距離分類自動'].isin(distance_filter)]
            filtered_df = filtered_df.sort_values(by='距離使用者(km)')

    if filtered_df.empty:
        return pd.DataFrame(), "查無結果", "", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.head(n_recommend)

    recommend_list = "\n".join([
        f"{i+1}. {row['名稱']} ({row['消費預算']} / {row['推薦時段']}) - {row['備註／特色']}，距離分類:{row.get('距離','')}，離你約:{row.get('距離使用者(km)','?'):.2f} km"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['名稱']}，價位{row['消費預算']}，特色：{row['備註／特色']}，距離分類:{row.get('距離','')}，離使用者:{row.get('距離使用者(km)','?'):.2f} km"
        for _, row in recommended_df.iterrows()
    ])

    map_html = generate_map(recommended_df)
    return recommended_df, recommend_list, ai_input, map_html

# --- Folium 地圖生成函式 ---
def generate_map(df_subset):
    df_valid = df_subset.dropna(subset=['緯度','經度'])
    if df_valid.empty:
        return "<p>⚠️ 無法顯示地圖：所有推薦地點都缺少座標。</p>"

    avg_lat = df_valid['緯度'].mean()
    avg_lng = df_valid['經度'].mean()
    fmap = folium.Map(location=[avg_lat, avg_lng], zoom_start=14)

    for _, row in df_valid.iterrows():
        name = row['名稱']
        info = f"{row['消費預算']} / {row['推薦時段']}<br>{row['備註／特色']}<br>距離分類:{row.get('距離','')}<br>離你約:{row.get('距離使用者(km)','?'):.2f} km"
        folium.Marker(
            [row['緯度'], row['經度']],
            tooltip=name,
            popup=folium.Popup(info, max_width=250)
        ).add_to(fmap)

    return fmap._repr_html_()

# --- AI 建議生成 (同步) ---
def generate_summary_sync(ai_input):
    if not ai_input:
        return "請先產生推薦清單。"

    system_prompt = (
        "你是一位幽默且效率高的台北玩樂顧問。"
        "請用繁體中文、輕鬆語氣，針對以下三個地點，分析優缺點，並建議使用者最適合去哪玩，約 100 字。"
    )
    user_query = f"三個玩樂地點資訊如下：{ai_input}。請給出分析與建議。"

    try:
        response = model.generate_content(contents=user_query, system_instruction=system_prompt)
        return response.text.strip()
    except:
        return "這三個地點都不錯，建議你先去距離最近、時間方便的那個！"

# --- Wrapper for Gradio ---
def wrapper_recommend_sync(price_filter, distance_filter, user_location):
    df_out, rec_text, ai_input, map_html = filter_and_recommend(df, price_filter, distance_filter, user_location)
    ai_summary = generate_summary_sync(ai_input)
    return df_out[['名稱','參考地點 (捷運站／商圈)','消費預算','推薦時段','距離']], rec_text, ai_summary, map_html

# --- Gradio 介面 ---
with gr.Blocks(title="玩樂地點推薦器 + 地圖") as demo:
    gr.Markdown("## 🎡 玩樂地點推薦器 (含地圖視覺化)")

    with gr.Row():
        price_select = gr.CheckboxGroup(choices=sorted(df['消費預算'].dropna().unique()), label="💰 消費預算")
        distance_select = gr.CheckboxGroup(choices=sorted(['短','中','長']), label="📍 距離篩選")
        user_location = gr.Textbox(label="📌 你的目前位置（可輸入地址或地點）", placeholder="例如：捷運大安站")
        recommend_btn = gr.Button("🎲 產生推薦")

    output_df = gr.DataFrame(label="推薦結果")
    output_text = gr.Textbox(label="推薦清單")
    output_summary = gr.Textbox(label="🤖 AI 決策建議", interactive=False)
    output_map = gr.HTML(label="🗺️ 推薦地圖")

    recommend_btn.click(
        fn=wrapper_recommend_sync,
        inputs=[price_select, distance_select, user_location],
        outputs=[output_df, output_text, output_summary, output_map]
    )

demo.launch(inbrowser=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60e36ca650b1607c1a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
